In [ ]:
import sys

if "google.colab" in sys.modules:
    !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
    !python rapidsai-csp-utils/colab/pip-install.py
    !pip install faiss-cpu

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 596, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 596 (delta 128), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (596/596), 195.77 KiB | 5.76 MiB/s, done.
Resolving deltas: 100% (302/302), done.
Installing RAPIDS remaining 25.04 libraries
Using Python 3.11.13 environment at: /usr
   Building cugraph-cu12==25.4.1
   Building cuproj-cu12==25.4.0
   Building cuspatial-cu12==25.4.0
   Building cuml-cu12==25.4.0
  × Failed to build `cuml-cu12==25.4.0`
  ├─▶ The build backend returned an error
  ╰─▶ Call to `wheel_stub.buildapi.build_wheel` failed (exit status: 1)

      [stderr]
        File
      "/root/.cache/uv/builds-v0/.tmpNRDP2X/lib/python3.11/site-packages/wheel_stub/wheel.py",
      line 249, in download_wheel
          return download_manual(wheel_directory, distribution, version,
      config)
                 ^^^^^^^^^^

In [ ]:
import cuvs
import faiss

cuvs_version = cuvs.__version__
faiss_cpu_version = faiss.__version__

print(f"cuVS version: {cuvs_version}")
print(f"FAISS CPU version: {faiss_cpu_version}")

cuVS version: 25.02.01
FAISS CPU version: 1.11.0


# 데이터 생성

In [ ]:
import numpy as np
import cupy as cp

# 1. NumPy 시드 고정
np.random.seed(42)

# 2. 데이터 생성 : 분포가 다른 2개의 2차원 데이터를 합쳐서 벡터 크기 차이가 나게 만듦
dataset_np = np.hstack([
    np.random.random((50000, 25)),   # [0, 1) 구간의 균일 분포
    np.random.randn(50000, 25) * 5    # 평균 0, 표준편차 5인 정규 분포
]).astype(np.float32)

# 3. Numpy 배열을 CuPy 배열로 변환
dataset_cp = cp.asarray(dataset_np)

# 4. 쿼리 개수 지정 (첫 번째 샘플을 쿼리로 사용)
n_queries = 1
query_np = dataset_np[:n_queries]
query_cp = dataset_cp[:n_queries]

# 5. 데이터셋과 쿼리 벡터의 일부 출력
print("데이터셋 일부:\n", dataset_np[:1], end='\n\n')  # 첫 2개 샘플만 출력
print("쿼리 벡터:\n", query_np)

데이터셋 일부:
 [[ 0.37454012  0.9507143   0.7319939   0.5986585   0.15601864  0.15599452
   0.05808361  0.8661761   0.601115    0.7080726   0.02058449  0.96990985
   0.83244264  0.21233912  0.18182497  0.1834045   0.30424225  0.52475643
   0.43194503  0.29122913  0.6118529   0.13949387  0.29214466  0.36636186
   0.45606998 -6.9410195  -0.7064649  -2.3158252   8.927271   -0.20885432
  -5.3248634  -2.2789273  -1.2751076   2.4537957   4.761805    5.567778
  -4.7681065   0.42192096  9.123224   -2.6262207   0.5368253  -8.330255
   1.1769367  -2.1636157   0.6136664   1.3033437  -1.3473666   1.8423494
   4.220154   11.622142  ]]

쿼리 벡터:
 [[ 0.37454012  0.9507143   0.7319939   0.5986585   0.15601864  0.15599452
   0.05808361  0.8661761   0.601115    0.7080726   0.02058449  0.96990985
   0.83244264  0.21233912  0.18182497  0.1834045   0.30424225  0.52475643
   0.43194503  0.29122913  0.6118529   0.13949387  0.29214466  0.36636186
   0.45606998 -6.9410195  -0.7064649  -2.3158252   8.927271   -0.20885

# FAISS IVF-Flat

In [ ]:
%%time

import numpy as np
import faiss

# 1. 코사인 유사도 계산에 필요한 L2 정규화 적용
faiss.normalize_L2(dataset_np)
faiss.normalize_L2(query_np)

# 2. 파라미터 설정 : Top-k, 클러스터 개수, 탐색할 클러스터 개수, 벡터 차원
k = 10
n_list = 1000
n_probes = 500
d = dataset_np.shape[1]

# 3. IVF-Flat 인덱스 생성 (내부는 내적 유사도)
quantizer = faiss.IndexFlatIP(d)   # 군집 중심용 인덱스
index = faiss.IndexIVFFlat(quantizer, d, n_list, faiss.METRIC_INNER_PRODUCT)

# 4. 인덱스 학습
index.train(dataset_np)

# 5. 인덱스에 벡터 추가
index.add(dataset_np)

# 6. 탐색할 군집 수 설정
index.nprobe = n_probes

# 7. 벡터 검색
distances, neighbors = index.search(query_np, k)

# 8. 검색 결과 출력
print("Top-k 인덱스:", neighbors[0])
print("유사도 점수 (cosine):", distances[0])

Top-k 인덱스: [    0 48906 22098 20005 43820 21295 17192  6777 14367  7147]
유사도 점수 (cosine): [1.         0.7389719  0.6744807  0.67263496 0.67114633 0.6699053
 0.6660686  0.65731394 0.6533355  0.65039325]
CPU times: user 3.13 s, sys: 20.8 ms, total: 3.15 s
Wall time: 4.91 s


# cuVS IVF-Flat

In [ ]:
%%time

import cupy as cp
from cuvs.neighbors import ivf_flat
from cuvs.neighbors.ivf_flat import IndexParams, SearchParams

# 1. 파라미터 설정 : Top-k, 클러스터 개수, 탐색할 클러스터 개수
k = 10
n_list = 1000
n_probes = 500

# 2. 인덱스 생성 파라미터 정의 (코사인 유사도 + 클러스터 개수)
params = IndexParams(
    n_lists=n_list,
    metric="cosine"
)

# 3. IVF-Flat 인덱스 생성
index = ivf_flat.build(params, dataset_cp)

# 4. 검색 파라미터 정의
search_params = SearchParams(n_probes=n_probes)

# 5. 벡터 검색
distances, neighbors = ivf_flat.search(search_params, index, query_cp, k)

# 6. CuPy 배열을 NumPy 배열로 변환
distances = cp.asnumpy(distances)
neighbors = cp.asnumpy(neighbors)

# 7. 검색 결과 출력
print("Top-k 인덱스:", neighbors[0])
print("유사도 점수 (cosine):", distances[0])

Top-k 인덱스: [    0 48906 22098 20005 43820 21295 17192  6777 14367  7147]
유사도 점수 (cosine): [0.         0.2610281  0.32551938 0.32736522 0.32885373 0.3300947
 0.33393133 0.34268612 0.3466646  0.34960675]
CPU times: user 539 ms, sys: 117 ms, total: 656 ms
Wall time: 1.67 s
